# Introduction

### **Here I have worked with a dummy restaurant dataset. My goal here is to provide recommendation of users to a restaurant. This is reverse approach to a traditional recommendation system. The Dataset Contains 16 features and a total of 100000 samples of data of various users referred by their ID.**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
%matplotlib notebook

from lightfm.data import Dataset
from lightfm import LightFM
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

# imports re for text cleaning 
import re
from datetime import datetime, timedelta

# we will ignore pandas warning 
import warnings
warnings.filterwarnings('ignore')

# Dataset Exploration

In [2]:
dataset = pd.read_csv('../input/dummy-data/train_100k.csv')
dataset

,customer_id,gender,latitude_x,longitude_x,latitude_y,longitude_y,vendor_category_en,delivery_charge,serving_distance,commission,delivery_available,discount_percentage,language,rank,restaurent_rating,restaurent_tag_name,restaurant_id
0,SZ5JI7X,male,-0.5050,0.0950,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106
1,WKYG878,male,0.2096,0.2810,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90
2,UXCWXNG,male,0.1357,-78.6000,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106
3,B9HSJBN,male,-0.8800,0.0755,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90
4,K3RGL6T,male,0.3879,0.5815,-0.1150,0.5460,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,949SKDG,male,0.6953,-78.7000,-1.0080,0.0792,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Fries,Mojitos",289
99996,E5BCNDB,male,-0.3720,-78.5600,1.7030,0.7600,Restaurants,0.7,10.0,0.0,Yes,0.0,EN,11,4.3,"Fresh Juices,Milkshakes,Mojitos ,Sandwiches,Sh...",192
99997,X18R5HU,male,-16.2800,-3.5460,0.0168,0.5479,Restaurants,0.7,15.0,0.0,Yes,35.0,EN,11,4.6,"Asian,Dimsum,Grills,Japanese,Rice,Soups",110
99998,LVRR6EE,male,0.4060,0.5884,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106


In [3]:
dataset.shape

(100000, 17)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   customer_id          100000 non-null  object 
 1   gender               100000 non-null  object 
 2   latitude_x           100000 non-null  float64
 3   longitude_x          100000 non-null  float64
 4   latitude_y           100000 non-null  float64
 5   longitude_y          100000 non-null  float64
 6   vendor_category_en   100000 non-null  object 
 7   delivery_charge      100000 non-null  float64
 8   serving_distance     100000 non-null  float64
 9   commission           100000 non-null  float64
 10  delivery_available   100000 non-null  object 
 11  discount_percentage  100000 non-null  float64
 12  language             100000 non-null  object 
 13  rank                 100000 non-null  int64  
 14  restaurent_rating    100000 non-null  float64
 15  restaurent_tag_nam

### Now lets see the state of the customers order and how many times a customer has made an order,

In [5]:
discounted = dataset['discount_percentage'] > 0.0
discounted.value_counts()

False    89404
True     10596
Name: discount_percentage, dtype: int64

### Now we see here that discount is not much of an issue as the customers who have made orders on account of providing discount is around 10%. However, this is particularly the case for this dataset and it might not be so for other real world dataset as people are more likely to order if they are given discount. But for this case we will ignore this characteristic.

# LightFM

## Useful Functions

In [6]:
def calculate_auc_score(lightfm_model, interactions_matrix, 
                        customer_features, restaurant_features):
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=customer_features, 
        user_features=restaurant_features, 
        num_threads=4).mean()
    return score

def generate_feature_list(dataframe, features_name):

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features

def generate_id(df, column_name, id_column):
    
    value_list = df[column_name].unique().tolist()
    #print(value_list)
    int_value_list = [i for i in range(len(value_list))]
    #print(int_value_list)
    id_list = []
    
    for i in df[column_name]:
        id_list.append(int_value_list[ value_list.index(i) ])
    
    df[id_column] = id_list

def create_weight(df, column_name, id_column):
    
    id_count_dict = dict()
    unique_id_list = df[column_name].unique().tolist()
    values_count = df[column_name].value_counts()
    new_feature = []
    
    for item in df[column_name]:
        
        new_feature.append(round(1/values_count.loc[item], 1))
    
    df[id_column] = new_feature

### Assigning Unique Values for the customer ID

In [7]:
generate_id(dataset, "customer_id", "id_customer")
dataset

,customer_id,gender,latitude_x,longitude_x,latitude_y,longitude_y,vendor_category_en,delivery_charge,serving_distance,commission,delivery_available,discount_percentage,language,rank,restaurent_rating,restaurent_tag_name,restaurant_id,id_customer
0,SZ5JI7X,male,-0.5050,0.0950,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,0
1,WKYG878,male,0.2096,0.2810,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,1
2,UXCWXNG,male,0.1357,-78.6000,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,2
3,B9HSJBN,male,-0.8800,0.0755,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,3
4,K3RGL6T,male,0.3879,0.5815,-0.1150,0.5460,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,949SKDG,male,0.6953,-78.7000,-1.0080,0.0792,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Fries,Mojitos",289,9070
99996,E5BCNDB,male,-0.3720,-78.5600,1.7030,0.7600,Restaurants,0.7,10.0,0.0,Yes,0.0,EN,11,4.3,"Fresh Juices,Milkshakes,Mojitos ,Sandwiches,Sh...",192,3623
99997,X18R5HU,male,-16.2800,-3.5460,0.0168,0.5479,Restaurants,0.7,15.0,0.0,Yes,35.0,EN,11,4.6,"Asian,Dimsum,Grills,Japanese,Rice,Soups",110,7749
99998,LVRR6EE,male,0.4060,0.5884,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,12067


In [8]:
dataset['serving_distance'].value_counts()

15.0    57805
6.0     10561
10.0    10492
8.0     10469
2.0      5380
5.0      5293
Name: serving_distance, dtype: int64

In [9]:
customer1 = dataset["restaurant_id"] == 43
customer1_dataset = dataset[customer1]
customer1_dataset #['serving_distance'].value_counts()

,customer_id,gender,latitude_x,longitude_x,latitude_y,longitude_y,vendor_category_en,delivery_charge,serving_distance,commission,delivery_available,discount_percentage,language,rank,restaurent_rating,restaurent_tag_name,restaurant_id,id_customer
4,K3RGL6T,male,0.3879,0.5815,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,4
13,1JGGTXJ,male,-0.6600,0.0846,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,13
19,VGAIVWE,male,0.6590,-78.7000,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,19
48,C3MNZ53,female,1.7930,0.7764,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,48
106,TWPHS2K,male,-0.6610,-78.5000,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99867,KT8GR94,male,0.1941,-78.6000,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,4295
99897,U37UY1Y,male,-0.2020,-78.5600,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,504
99913,HUQB4OY,female,-0.0410,-0.0560,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,5157
99985,4L9M7ZJ,male,-1.4360,0.0751,-0.115,0.546,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,14204


In [10]:
dataset['customer_id'].value_counts()

X58BXH0    66
O86521G    56
YYKZN0F    52
9VJZUWB    48
YQC2R6H    47
           ..
9JDO2LM     1
F8O7YJH     1
8HN81JL     1
ZX3HBSJ     1
J4RHVRN     1
Name: customer_id, Length: 15267, dtype: int64

In [11]:
create_weight(dataset, "id_customer", "customer_order_weight")
dataset

,customer_id,gender,latitude_x,longitude_x,latitude_y,longitude_y,vendor_category_en,delivery_charge,serving_distance,commission,delivery_available,discount_percentage,language,rank,restaurent_rating,restaurent_tag_name,restaurant_id,id_customer,customer_order_weight
0,SZ5JI7X,male,-0.5050,0.0950,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,0,0.2
1,WKYG878,male,0.2096,0.2810,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,1,0.1
2,UXCWXNG,male,0.1357,-78.6000,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,2,0.1
3,B9HSJBN,male,-0.8800,0.0755,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,3,0.1
4,K3RGL6T,male,0.3879,0.5815,-0.1150,0.5460,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,4,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,949SKDG,male,0.6953,-78.7000,-1.0080,0.0792,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Fries,Mojitos",289,9070,0.2
99996,E5BCNDB,male,-0.3720,-78.5600,1.7030,0.7600,Restaurants,0.7,10.0,0.0,Yes,0.0,EN,11,4.3,"Fresh Juices,Milkshakes,Mojitos ,Sandwiches,Sh...",192,3623,0.1
99997,X18R5HU,male,-16.2800,-3.5460,0.0168,0.5479,Restaurants,0.7,15.0,0.0,Yes,35.0,EN,11,4.6,"Asian,Dimsum,Grills,Japanese,Rice,Soups",110,7749,0.1
99998,LVRR6EE,male,0.4060,0.5884,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,12067,0.2


In [12]:
restaurant_feature_list = generate_feature_list(
    dataset,
    ['restaurent_tag_name'])
restaurant_feature_list

0              American
1               Burgers
2         Free Delivery
3              Hot Dogs
4                 Pasta
              ...      
532556    Free Delivery
532557         Hot Dogs
532558            Pasta
532559         Desserts
532560          Mexican
Length: 532561, dtype: object

In [13]:
customer_feature_list = generate_feature_list(
    dataset,
    ['vendor_category_en'])
customer_feature_list

0        Restaurants
1        Restaurants
2        Restaurants
3        Restaurants
4        Restaurants
            ...     
99995    Restaurants
99996    Restaurants
99997    Restaurants
99998    Restaurants
99999    Restaurants
Length: 100000, dtype: object

In [14]:
temp_list = []
temp_count = 0
for i in dataset['restaurent_tag_name']:
    temp_list.append( (dataset['restaurant_id'].iloc[temp_count], i.split(',') ) )
    temp_count = temp_count+1
    
dataset['restaurant_feature_list'] = temp_list
dataset

,customer_id,gender,latitude_x,longitude_x,latitude_y,longitude_y,vendor_category_en,delivery_charge,serving_distance,commission,delivery_available,discount_percentage,language,rank,restaurent_rating,restaurent_tag_name,restaurant_id,id_customer,customer_order_weight,restaurant_feature_list
0,SZ5JI7X,male,-0.5050,0.0950,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,0,0.2,"(106, [American, Burgers, Free Delivery, Hot D..."
1,WKYG878,male,0.2096,0.2810,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,1,0.1,"(90, [American, Burgers, Desserts, Mojitos , P..."
2,UXCWXNG,male,0.1357,-78.6000,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,2,0.1,"(106, [American, Burgers, Free Delivery, Hot D..."
3,B9HSJBN,male,-0.8800,0.0755,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,3,0.1,"(90, [American, Burgers, Desserts, Mojitos , P..."
4,K3RGL6T,male,0.3879,0.5815,-0.1150,0.5460,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,4,0.1,"(43, [American, Burgers, Fries, Sandwiches])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,949SKDG,male,0.6953,-78.7000,-1.0080,0.0792,Restaurants,0.7,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Fries,Mojitos",289,9070,0.2,"(289, [American, Burgers, Fries, Mojitos ])"
99996,E5BCNDB,male,-0.3720,-78.5600,1.7030,0.7600,Restaurants,0.7,10.0,0.0,Yes,0.0,EN,11,4.3,"Fresh Juices,Milkshakes,Mojitos ,Sandwiches,Sh...",192,3623,0.1,"(192, [Fresh Juices, Milkshakes, Mojitos , San..."
99997,X18R5HU,male,-16.2800,-3.5460,0.0168,0.5479,Restaurants,0.7,15.0,0.0,Yes,35.0,EN,11,4.6,"Asian,Dimsum,Grills,Japanese,Rice,Soups",110,7749,0.1,"(110, [Asian, Dimsum, Grills, Japanese, Rice, ..."
99998,LVRR6EE,male,0.4060,0.5884,0.6187,0.5273,Restaurants,0.0,15.0,0.0,Yes,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,12067,0.2,"(106, [American, Burgers, Free Delivery, Hot D..."


In [15]:
temp_list = []
temp_count = 0
for i in dataset['vendor_category_en']:
    temp_list.append( (dataset['id_customer'].iloc[temp_count], [i] ) )
    temp_count = temp_count+1
    
dataset['customer_feature_list'] = temp_list
dataset

,customer_id,gender,latitude_x,longitude_x,latitude_y,longitude_y,vendor_category_en,delivery_charge,serving_distance,commission,...,discount_percentage,language,rank,restaurent_rating,restaurent_tag_name,restaurant_id,id_customer,customer_order_weight,restaurant_feature_list,customer_feature_list
0,SZ5JI7X,male,-0.5050,0.0950,0.6187,0.5273,Restaurants,0.0,15.0,0.0,...,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,0,0.2,"(106, [American, Burgers, Free Delivery, Hot D...","(0, [Restaurants])"
1,WKYG878,male,0.2096,0.2810,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,...,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,1,0.1,"(90, [American, Burgers, Desserts, Mojitos , P...","(1, [Restaurants])"
2,UXCWXNG,male,0.1357,-78.6000,0.6187,0.5273,Restaurants,0.0,15.0,0.0,...,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,2,0.1,"(106, [American, Burgers, Free Delivery, Hot D...","(2, [Restaurants])"
3,B9HSJBN,male,-0.8800,0.0755,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,...,0.0,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,3,0.1,"(90, [American, Burgers, Desserts, Mojitos , P...","(3, [Restaurants])"
4,K3RGL6T,male,0.3879,0.5815,-0.1150,0.5460,Restaurants,0.7,15.0,0.0,...,0.0,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,4,0.1,"(43, [American, Burgers, Fries, Sandwiches])","(4, [Restaurants])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,949SKDG,male,0.6953,-78.7000,-1.0080,0.0792,Restaurants,0.7,15.0,0.0,...,0.0,EN,11,4.5,"American,Burgers,Fries,Mojitos",289,9070,0.2,"(289, [American, Burgers, Fries, Mojitos ])","(9070, [Restaurants])"
99996,E5BCNDB,male,-0.3720,-78.5600,1.7030,0.7600,Restaurants,0.7,10.0,0.0,...,0.0,EN,11,4.3,"Fresh Juices,Milkshakes,Mojitos ,Sandwiches,Sh...",192,3623,0.1,"(192, [Fresh Juices, Milkshakes, Mojitos , San...","(3623, [Restaurants])"
99997,X18R5HU,male,-16.2800,-3.5460,0.0168,0.5479,Restaurants,0.7,15.0,0.0,...,35.0,EN,11,4.6,"Asian,Dimsum,Grills,Japanese,Rice,Soups",110,7749,0.1,"(110, [Asian, Dimsum, Grills, Japanese, Rice, ...","(7749, [Restaurants])"
99998,LVRR6EE,male,0.4060,0.5884,0.6187,0.5273,Restaurants,0.0,15.0,0.0,...,0.0,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,12067,0.2,"(106, [American, Burgers, Free Delivery, Hot D...","(12067, [Restaurants])"


In [16]:
dataset['interaction_tuple'] = list(
                                zip(
                                    dataset.restaurant_id,
                                    dataset.id_customer,
                                    dataset.customer_order_weight
                                )
                            )
dataset

,customer_id,gender,latitude_x,longitude_x,latitude_y,longitude_y,vendor_category_en,delivery_charge,serving_distance,commission,...,language,rank,restaurent_rating,restaurent_tag_name,restaurant_id,id_customer,customer_order_weight,restaurant_feature_list,customer_feature_list,interaction_tuple
0,SZ5JI7X,male,-0.5050,0.0950,0.6187,0.5273,Restaurants,0.0,15.0,0.0,...,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,0,0.2,"(106, [American, Burgers, Free Delivery, Hot D...","(0, [Restaurants])","(106, 0, 0.2)"
1,WKYG878,male,0.2096,0.2810,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,...,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,1,0.1,"(90, [American, Burgers, Desserts, Mojitos , P...","(1, [Restaurants])","(90, 1, 0.1)"
2,UXCWXNG,male,0.1357,-78.6000,0.6187,0.5273,Restaurants,0.0,15.0,0.0,...,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,2,0.1,"(106, [American, Burgers, Free Delivery, Hot D...","(2, [Restaurants])","(106, 2, 0.1)"
3,B9HSJBN,male,-0.8800,0.0755,-0.6010,0.0960,Restaurants,0.0,15.0,0.0,...,EN,1,4.4,"American,Burgers,Desserts,Mojitos ,Pasta",90,3,0.1,"(90, [American, Burgers, Desserts, Mojitos , P...","(3, [Restaurants])","(90, 3, 0.1)"
4,K3RGL6T,male,0.3879,0.5815,-0.1150,0.5460,Restaurants,0.7,15.0,0.0,...,EN,11,4.3,"American,Burgers,Fries,Sandwiches",43,4,0.1,"(43, [American, Burgers, Fries, Sandwiches])","(4, [Restaurants])","(43, 4, 0.1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,949SKDG,male,0.6953,-78.7000,-1.0080,0.0792,Restaurants,0.7,15.0,0.0,...,EN,11,4.5,"American,Burgers,Fries,Mojitos",289,9070,0.2,"(289, [American, Burgers, Fries, Mojitos ])","(9070, [Restaurants])","(289, 9070, 0.2)"
99996,E5BCNDB,male,-0.3720,-78.5600,1.7030,0.7600,Restaurants,0.7,10.0,0.0,...,EN,11,4.3,"Fresh Juices,Milkshakes,Mojitos ,Sandwiches,Sh...",192,3623,0.1,"(192, [Fresh Juices, Milkshakes, Mojitos , San...","(3623, [Restaurants])","(192, 3623, 0.1)"
99997,X18R5HU,male,-16.2800,-3.5460,0.0168,0.5479,Restaurants,0.7,15.0,0.0,...,EN,11,4.6,"Asian,Dimsum,Grills,Japanese,Rice,Soups",110,7749,0.1,"(110, [Asian, Dimsum, Grills, Japanese, Rice, ...","(7749, [Restaurants])","(110, 7749, 0.1)"
99998,LVRR6EE,male,0.4060,0.5884,0.6187,0.5273,Restaurants,0.0,15.0,0.0,...,EN,11,4.5,"American,Burgers,Free Delivery,Hot Dogs,Pasta",106,12067,0.2,"(106, [American, Burgers, Free Delivery, Hot D...","(12067, [Restaurants])","(106, 12067, 0.2)"


In [17]:
model_dataset = Dataset()
model_dataset.fit(
    set(dataset['restaurant_id']),
    set(dataset['id_customer'])
    #item_features=customer_feature_list,
    #user_features=restaurant_feature_list
)

In [18]:
interactions, weights = model_dataset.build_interactions(
    dataset['interaction_tuple']
)

In [19]:
customer_features = model_dataset.build_item_features(
    dataset['customer_feature_list']
)
restaurant_features = model_dataset.build_user_features(
    dataset['restaurant_feature_list']
)

ValueError: Feature Restaurants not in feature mapping. Call fit first.

In [20]:
model = LightFM(
    no_components=150,
    learning_rate=0.05,
    loss='warp',
    random_state=2019)

model.fit(
    interactions,
    item_features=customer_features,
    user_features=restaurant_features,
    sample_weight=weights,
    epochs=10, num_threads=4, verbose=True)

NameError: name 'customer_features' is not defined

In [21]:
calculate_auc_score(model, interactions, customer_features, restaurant_features)

NameError: name 'customer_features' is not defined

In [22]:
customer_features

NameError: name 'customer_features' is not defined

In [23]:
restaurant_features

NameError: name 'restaurant_features' is not defined

In [24]:
customers_id_for_predict = dataset['id_customer'].values.tolist()
scores = model.predict(
            106,
            customers_id_for_predict,
            item_features=customer_features,
            user_features=restaurant_features
        )
scores

NameError: name 'customer_features' is not defined